## Loading Data and Data integration
in this notebook we load and create our dataset that we'll use for the rest of the project

In [1]:
import pandas as pd

## Loading the data set

Read the .csv file containing the data. The first line contains the list of attributes. The data is assigned to a Pandas dataframe.

In [2]:
df = pd.read_csv('tennis_matches.csv', index_col = 0)#, sep=',', index_col=0) #alternative in case of json source df.to_json('filename.json')
print(df.head()) #print the first records of a df, 
# print()
# print(df.tail()) #print the last records of a df.

  tourney_id tourney_name surface  draw_size tourney_level  tourney_date  \
0  2019-M020     Brisbane    Hard       32.0             A    20181231.0   
1  2019-M020     Brisbane    Hard       32.0             A    20181231.0   
2  2019-M020     Brisbane    Hard       32.0             A    20181231.0   
3  2019-M020     Brisbane    Hard       32.0             A    20181231.0   
4  2019-M020     Brisbane    Hard       32.0             A    20181231.0   

   match_num  winner_id winner_entry         winner_name  ... l_2ndWon  \
0      300.0   105453.0          NaN       Kei Nishikori  ...     20.0   
1      299.0   106421.0          NaN     Daniil Medvedev  ...      7.0   
2      298.0   105453.0          NaN       Kei Nishikori  ...      6.0   
3      297.0   104542.0           PR  Jo-Wilfried Tsonga  ...      9.0   
4      296.0   106421.0          NaN     Daniil Medvedev  ...     19.0   

   l_SvGms l_bpSaved  l_bpFaced  winner_rank winner_rank_points loser_rank  \
0     14.0      10.0

# Data Integration
We load another table containing the name and surname of male and female people, we eliminate duplicates, mark NaN genders as 'U' and concatenate Female and Male tables in a unique table called table_gender

In [3]:
df_male = pd.read_csv('male_players.csv')#, sep=',') 
df_male.dropna(inplace=True)
df_male_names = pd.DataFrame()
df_male_names['Name'] = df_male['name'].astype(str) + " " + df_male['surname'].astype(str)
df_male_names.drop_duplicates(inplace=True)

#print(df_male_names.isna().sum())
#print(df_male_names.isnull().sum())
#print(df_male_names.describe())
#print(df_male_names)

df_male_names.insert(1, 'gender', 'M', allow_duplicates=True)
#print(df_male_names)

df_female = pd.read_csv('female_players.csv')#, sep=',') 
df_female.dropna(inplace=True)
df_female_names = pd.DataFrame()
df_female_names['Name'] = df_female['name'].astype(str) + " " + df_female['surname'].astype(str)
df_female_names.drop_duplicates(inplace=True)
df_female_names.insert(1, 'gender', 'F', allow_duplicates=True)
#print(df_female_names)


result = pd.concat([df_male_names, df_female_names]) #we create a new table to have the global view
#print(result.duplicated(subset=['Name']).any())
#print(result.describe())

pd.options.mode.chained_assignment = None 

duplicateDFRow = result[result.duplicated(['Name'], keep=False)]
duplicateDFRow.drop_duplicates(subset=['Name'], inplace=True)
duplicateDFRow['gender'].replace({"M": 'U'},inplace=True)

#print(result.describe())
#print(duplicateDFRow.describe())

table_gender = pd.concat([result, duplicateDFRow]) 

table_gender.drop_duplicates(subset=['Name'], keep='last', inplace=True)

#print(table_gender['gender'].describe())

Then we join the matchs's table with the table_gender

In [4]:
#df.drop_duplicates(inplace = True)
matches = df
#print(tmp.info())
matches = matches.join(table_gender.set_index('Name'), on='winner_name')
matches.rename(columns={'gender': 'winner_gender'}, inplace = True)
#print(tmp['winner_name'].head(), tmp['gender'].head())
matches = matches.join(table_gender.set_index('Name'), on='loser_name')
matches.rename(columns={'gender': 'loser_gender'}, inplace = True)
matches.to_csv('matches_with_gender.csv')
#print(tmp.head())
#print(tmp.info())